# About the project
The end goal of this project is to classify patients with high protein concentration in urine and the healthy group based on SERS (Surface Enchanced Raman Spectroscopy) spectral data and biomedical data.  
This project is to be released as a research paper later in 2022 or 2023. Some information might not be fully shown here as a result.

The project is divided into several Jupyter notebooks with the following names:
1) Import raw urine spectra (part 1)
2) Spectra processing (part 2)
3) Classification of patients (part 3)
4) Biomedical data (part 4)
5) Comparison of nanoparticles (part 5)

Author of all codes: Sultan Aitekenov, sultanaitekenov@gmail.com

Part of the upcoming abstract:
Excessive protein excretion in human urine is an early and sensitive marker of diabetic nephropathy, primary and secondary renal disease. Kidney problems, particularly chronic kidney disease, remain among the few growing causes of mortality in the world. Therefore, it is important to develop efficient, expressive, and low-cost method for protein determination. Surface enhanced Raman spectroscopy (SERS) methods are potential candidates to achieve those criteria. In this paper, the SERS methods was developed to distinguish patients with proteinuria and the healthy group. Commercial gold nanoparticles with the diameter of 60 nm and 100 nm, and silver nanoparticles with the diameter of 100 nm were employed. Silver, gold, silicon and test slides covered with aluminium tape were utilized as substrates. Obtained spectra were analysed with several machine learning algorithms coupled with the PCA, ROC curve, and cross-validation methods. 

# Classification of patients (part 3)

## Import data

### Import modules

In [ ]:
# other modules related to classification are imported later
import pandas as pd
import numpy as np
import copy
import pickle
import matplotlib.pyplot as plt

### Raman Shift

In [ ]:
# Si_60nm_AuNPs is not included though
raman_shift_400_1800=np.array(pd.read_csv('raman_shift_400_1800.csv', header=None))
raman_shift_600_1800=np.array(pd.read_csv('raman_shift_600_1800.csv', header=None))
wave = raman_shift_400_1800[0]
wave_Si = raman_shift_600_1800[0]

### Processed urine spectra

In [ ]:
# data contains a nested dictionary
f = open('processed_urine_spectra.pkl', 'rb')
processed_urine_spectra = pickle.load(f)

In [ ]:
processed_urine_spectra.keys()

In [ ]:
# delete 'Au_40nm_AuNPs' as it was not used for classification, see part 5
del processed_urine_spectra['Au_40nm_AuNPs']
del processed_urine_spectra['Au_no_AuNPs']
del processed_urine_spectra['Au_HSA_AuNPs']
del processed_urine_spectra['glass_no_AuNPs']
del processed_urine_spectra['Si_no_AuNPs']

### Protein data and health status
Both protein data and health status will be used for labelling. They do not coincide with each other. To get more information about biomedical data see Part 5.

#### Protein data

In [ ]:
# protein data
bio_data = pd.read_csv('urine biomedicals processed.csv')
bio_data.head(10)

In [ ]:
# convert info_protein to dictionary with keys being equal to patients' ID for all patients in the database 
info_protein={}
for i in range(0,len(bio_data)):
    key_ID = bio_data['patient_ID'][i]
    info_protein[key_ID] = float(bio_data['Protein, mg/L'][i]) #converts to mg/L

#### Health status

In [ ]:
bio_data['status'].unique()

In [ ]:
(bio_data['status'] == 'diseased').sum()

In [ ]:
(bio_data['status'] == 'healthy').sum()

In [ ]:
bio_data['status'].count()

In [ ]:
# health status
health_status={}
for i in range(0,len(bio_data)):
    key_ID = bio_data['patient_ID'][i]
    if bio_data['status'][i] == 1:
        health_status[key_ID] = 1
    elif bio_data['status'][i] == 0:
        health_status[key_ID] = 0

## Assign patients to high or low protein group

### Define function that assigns to high and low protein concentration groups

In [ ]:
# function for creation of groups, the same structure as spectra data
def raman_protein_info(input_spectra, input_protein, protein_threshold):
    """
    Compares samples with database. Returns error if match is not found for
    samples. Reports which samples were not included.
    low protein == 0
    high protein == 1
    protein threshold in mg/L
    
    Output file has the same structure as input_spectra
    """
    import copy
    
    # copies the same structure as input spectra
    output_group = copy.deepcopy(input_spectra)
    
    # cycle through expiremntal sets
    for key_set in input_spectra.keys():
        # cycles through patients with key_ID
        for key_ID in input_spectra[key_set].keys():
            # take protein gramm from input_protein
            protein_gramm = input_protein.get(key_ID)
            if protein_gramm == None:
                print(f"error - your protein gramm value is not found in the database for patient {key_ID} in {key_set}")
                # delete patients who is not in the protein database
                output_group[key_set][key_ID] = None
                # del input_spectra[key_set][key_ID]
            elif protein_gramm >= protein_threshold:
                output_group[key_set][key_ID] = 1
            elif protein_gramm <= protein_threshold:
                output_group[key_set][key_ID] = 0
    
    
    output_group_final = output_group
    return output_group_final

### For each experimental set assign to protein groups

In [ ]:
# protein is set to 300 mg/L as it is important threshold for medical diagnostics
protein_threshold =  300
groups_protein = raman_protein_info(processed_urine_spectra, info_protein, protein_threshold)
groups_protein;

### Define function that assigns according to the health status

In [ ]:
def raman_health_info(input_spectra, health_status):
    """
    Compares samples with database according to their health status.
    diseased == 1
    healthy == 0

    Output file has the same structure as input_spectra
    """
    import copy
    
    # copies the same structure as input spectra
    output_group = copy.deepcopy(input_spectra)
    
    # cycle through expiremntal sets
    for key_set in input_spectra.keys():
        # cycles through patients with key_ID
        for key_ID in input_spectra[key_set].keys():
            
            output_group[key_set][key_ID] = health_status.get(key_ID)

    return output_group

### For each experimental set assign to health status

In [ ]:
groups_health = raman_health_info(processed_urine_spectra, health_status)

## Prepare data for x and y training sets

### Select a limited range in raman shift, as it might help to improve classification scores. Uncomment or change status of the cells to Code.

### Convert dictionaries to matrices

In [ ]:
# create empty dict
x = {}
y = {}
IDs = {}


for key_set in processed_urine_spectra.keys():
        # create empty matrix to assign to them values later
        matrix_x = []
        matrix_y = []
        matrix_IDs = []
        
        # loop to make dict into matrix
        for key_ID in processed_urine_spectra[key_set].keys():                                                      
            matrix_x.append( processed_urine_spectra[key_set][key_ID] )
            # matrix_y.append( groups_health[key_set][key_ID] )
            matrix_y.append( groups_protein[key_set][key_ID] )
            matrix_IDs.append( int(key_ID) )
            
        # assign matrix to x values    
        x[key_set] = np.array(matrix_x)
        y[key_set] = np.array(matrix_y)
        IDs[key_set] = np.array(matrix_IDs)

## Spectra high vs low protein groups

In [ ]:
# check whether length of x and y agrees in each key_set
for key_set in x:
    print(len(x[key_set]), len(y[key_set])) 

In [ ]:
y['Ag_100nm_AgNPs'][1]

In [ ]:
x['Ag_100nm_AgNPs'][1]

In [ ]:
# create dictionaries x_low and x_high
x_low = dict.fromkeys( list(x.keys()), [])
x_high = dict.fromkeys( list(x.keys()), [] )

print(x_low)
print(x_high)

In [ ]:
# check whether we have correct keys
x.keys() == y.keys() == x_low.keys() == x_high.keys()

In [ ]:
# append values to x_low and x_high
for key_set in x.keys():
    
    x_low[key_set] = []
    x_high[key_set] = []
    
    for i in range(len(x[key_set])):
        if y[key_set][i] == 0:
            x_low[key_set].append(x[key_set][i])
        elif y[key_set][i] == 1:
            x_high[key_set].append(x[key_set][i])

In [ ]:
# check 
for key_set in x:
    print(len(x_low[key_set]), len(x_high[key_set]), len(x[key_set]))
    print(len(x_low[key_set]) + len(x_high[key_set]) == len(x[key_set]), key_set)

In [ ]:
key_set = 'Ag_100nm_AgNPs'

In [ ]:
len(x_low[key_set][1])

In [ ]:
# visualize spectra
for key_set in x:
    plt.figure(figsize =(16,8))
    if key_set != 'Si_60nm_AuNPs' and key_set != 'Si_3x_100nm_AuNPs':
        for i in range(len(x_low[key_set])):
            plt.subplot(1,2,1)
            # print(key_set,len(wave), len(x_low[key_set][i]))
            plt.plot(wave, x_low[key_set][i])
            plt.xlabel('Raman shift, cm-1')
            plt.ylabel('Raman intensity, a.u.')
            plt.title(f'{key_set} - low protein')
        for i in range(len(x_high[key_set])):
            plt.subplot(1,2,2)
            plt.plot(wave, x_high[key_set][i])
            plt.xlabel('Raman shift, cm-1')
            plt.ylabel('Raman intensity, a.u.')
            plt.title(f'{key_set} - high protein')
        
    else:
        for i in range(len(x_low[key_set])):
            plt.subplot(1,2,1)
            plt.plot(wave_Si, x_low[key_set][i])
            plt.xlabel('Raman shift, cm-1')
            plt.ylabel('Raman intensity, a.u.')
            plt.title(f'{key_set} - low protein')
        for i in range(len(x_high[key_set])):
            plt.subplot(1,2,2)
            plt.plot(wave_Si, x_high[key_set][i])
            plt.xlabel('Raman shift, cm-1')
            plt.ylabel('Raman intensity, a.u.')
            plt.title(f'{key_set} - high protein')

In [ ]:
x_low_average = dict.fromkeys( list(x.keys()) , [])
x_high_average = dict.fromkeys( list(x.keys()) , [])

for key_set in x_low:
    
    x_low_average[key_set] = []
    x_high_average[key_set] = []
    
    
    x_low_average[key_set].append(np.mean(x_low[key_set], axis = 0))
    x_high_average[key_set].append(np.mean(x_high[key_set], axis = 0))


In [ ]:
x_low_average

In [ ]:
len(x_low_average['Ag_100nm_AgNPs'][0])

In [ ]:
list(x.keys())

In [ ]:
plt.figure(figsize=(16,8))

for i, key_set in zip([1,2], ['Si_60nm_AuNPs', 'Si_3x_100nm_AuNPs']):
    
    plt.subplot(1,2,i)
    
    plt.plot(wave_Si, x_low_average[key_set][0], label = 'low protein')
    plt.plot(wave_Si, x_high_average[key_set][0], label = 'high protein')
    plt.plot(wave_Si, x_high_average[key_set][0]-x_low_average[key_set][0], label = 'high - low')
    plt.xlabel('Raman shift, cm-1')
    plt.ylabel('Raman intensity, a.u.')
    plt.title(f'{key_set}')

    
plt.figure(figsize=(16,24))

for i, key_set in zip([1,2,3,4,5,6], ['Ag_100nm_AgNPs', 'Ag_100nm_AuNPs', 'Au_60nm_AuNPs', 'Au_100nm_AuNPs','Al_tape_60nm_AuNPs', 'Al_tape_100nm_AuNPs', ]):
    
    plt.subplot(3,2,i)
    
    plt.plot(wave, x_low_average[key_set][0], label = 'low protein')
    plt.plot(wave, x_high_average[key_set][0], label = 'high protein')
    plt.plot(wave, x_high_average[key_set][0]-x_low_average[key_set][0], label = 'high - low')
    plt.xlabel('Raman shift, cm-1')
    plt.ylabel('Raman intensity, a.u.')
    plt.title(f'{key_set}')


## Save x and y

In [ ]:
file_excel = 'processed_urine_spectra_X.xlsx'

writer = pd.ExcelWriter(file_excel)

for key_set in x:
    
    df = pd.DataFrame.from_dict( x[key_set] )
    df.to_excel(writer, sheet_name=key_set, header=False, index=False)

writer.save()
# writer.close()

In [ ]:
file_excel = 'processed_urine_spectra_Y.xlsx'

writer = pd.ExcelWriter(file_excel)

for key_set in y:
    
    df = pd.DataFrame.from_dict( y[key_set] )
    df.to_excel(writer, sheet_name=key_set, header=False, index=False)

writer.save()
# writer.close()

# Classification

All metrics will run as dependants of PC components. Exactly as in paper 1. Besides LDA, I will add other models.

## PCA 

In [ ]:
# PCA
from sklearn.decomposition import PCA

# perform PCA analysis on all set
pca = PCA(0.999)
x_pca = copy.deepcopy(x)
for key_set in x.keys():
    x_pca[key_set] = pca.fit(x[key_set]).transform(x[key_set]) # xx_pca=pca.fit_transform(x)
    print(f'{key_set}' + f' - PCA components {pca.n_components_}\n')

In [ ]:
# example of how to take first 3 PC components
x_pca['Ag_100nm_AgNPs'][1][:3]

## Estimators preparation

In [ ]:
# Estimators:
# LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# logistic regression
from sklearn.linear_model import LogisticRegression
# KNN
from sklearn.neighbors import KNeighborsClassifier
# SVM
from sklearn.svm import SVC
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
# RFC
from sklearn.ensemble import RandomForestClassifier
# Naive Gaussian
from sklearn.naive_bayes import GaussianNB

# create dictionary of models
models_dict = {#'LogisticRegression': LogisticRegression(),
              'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
               #'Gaussian Naive Bayes': GaussianNB(),
              'K-nearest Neighbors': KNeighborsClassifier(n_neighbors=10),
              'Support Vector Machnines': SVC()}

# create dict of scores that will be overwritten
empty_models_dict = copy.deepcopy(models_dict)
for key_model in empty_models_dict.keys():
    empty_models_dict[key_model] = []

# scores dict need to have key_set
results_acc_cv = {}
results_acc = {}
probability_dict = {}
probability_dict_cv = {}
auc_values = {}
auc_values_cv = {}
for key_set in x.keys():
    results_acc_cv[key_set] = copy.deepcopy(empty_models_dict)
    results_acc[key_set] = copy.deepcopy(empty_models_dict)
    probability_dict[key_set] = copy.deepcopy(empty_models_dict)
    probability_dict_cv[key_set] = copy.deepcopy(empty_models_dict)
    auc_values[key_set] = copy.deepcopy(empty_models_dict)
    auc_values_cv[key_set] = copy.deepcopy(empty_models_dict)

In [ ]:
# see how it looks
results_acc_cv

## Training

In [ ]:
# define range for PC components
PC_range = np.arange(3,25,2)
PC_range =np.append(PC_range, [50, 60, 70])
PC_range

In [ ]:
# model training and evaluation on all data points
from sklearn import metrics

for key_set in results_acc_cv.keys():
    
    for key_model in results_acc_cv[key_set].keys():
        
        # cycle through each PC component
        for i in PC_range:
            
            # define some variables
            model = models_dict[key_model]
            x_ = x_pca[key_set][:,:i]
                        
            # fit, predict
            model.fit(x_, y[key_set])            
            y_pred = model.predict(x_)
            
            acc = metrics.accuracy_score(y[key_set], y_pred)
            
            acc = round(acc, 4)
            
            results_acc[key_set][key_model].append(acc)

## Crossvalidation

In [ ]:
# explore x_pca
print(x_pca.keys())
print(x_pca['Ag_100nm_AgNPs'].shape)

In [ ]:
from sklearn.model_selection import cross_val_score

K = 10

for key_set in results_acc_cv.keys():
    
    for key_model in results_acc_cv[key_set].keys():
        
        # cycle through each PC component
        for i in PC_range:
            # print(key_set, key_model, i)
            acc_cv_array = cross_val_score(                
                models_dict[key_model],
                x_pca[key_set][:,:i], # [:, :3] selects first 3 columns for all arrays
                y[key_set],
                scoring='accuracy',
                cv=K
            )
            # print(acc_cv_array)
            
            acc_cv_array = np.round_(acc_cv_array, decimals=4)
            
            results_acc_cv[key_set][key_model].append([acc_cv_array])

## Create means for results_acc_cv

In [ ]:
results_acc_cv[key_set]['Linear Discriminant Analysis']

In [ ]:
# create a deepcopy
results_acc_mean_cv = copy.deepcopy(results_acc_cv)

# calculate results_acc_mean_cv
for key_set in results_acc_cv.keys():
    
    for key_model in results_acc_cv[key_set].keys():
        
        for i in range( len(results_acc_cv[key_set][key_model]) ):
            
            results_acc_mean_cv[key_set][key_model][i] = np.mean(results_acc_cv[key_set][key_model][i])

In [ ]:
results_acc_mean_cv.keys()

## Compare accuracies from cross validation

## Visualize tables

In [ ]:
# take example set
key_set = 'Ag_100nm_AuNPs'

In [ ]:
df_cv = pd.DataFrame.from_dict(results_acc_mean_cv[key_set])
df_cv['PC_score'] = PC_range
df_cv

In [ ]:
df = pd.DataFrame.from_dict(results_acc[key_set])
df['PC_score'] = PC_range
df

## Save df tables to excel

In [ ]:
file_excel = 'summary tables ACC.xlsx'

writer = pd.ExcelWriter(file_excel)

for key_set in results_acc:
    
    # no crossvalidation
    df = pd.DataFrame.from_dict( results_acc[key_set] )
    
    # crossvalidation
    df_cv = pd.DataFrame.from_dict( results_acc_mean_cv[key_set] )
    
    # merge
    df_merged = pd.concat([df, df_cv], axis =1)
    
    # insert PC component at the first place
    df_merged.insert(0, 'PC_component', PC_range)
    
    # save to excel
    df_merged.to_excel(writer, sheet_name=key_set, index=False)

writer.save()
writer.close()

In [ ]:
df_merged

# Learning curves

In [ ]:
# Learning curve for Ag_100nm_AuNPs as it has the best performance
from sklearn.model_selection import learning_curve

# plot learning curve 

key_set = 'Ag_100nm_AuNPs'
train_sizes, train_scores, valid_scores = learning_curve(KNeighborsClassifier(), x_pca[key_set], y[key_set], train_sizes = np.linspace(0.1, 1.0, 5), scoring = 'accuracy', cv=int(K))
train_scores_mean = train_scores.mean(axis = 1)
valid_scores_mean = valid_scores.mean(axis = 1)
plt.figure(figsize = [15,10])
plt.plot(train_sizes, train_scores_mean, "o--", label = 'training scores')
plt.plot(train_sizes, valid_scores_mean, "o--", label = 'validation scores')
plt.legend()
plt.ylabel('accuracy')
plt.xlabel('training size')


In [ ]:
# Learning curve for Al_tape_60nm_AuNPs as it has the worst performance
from sklearn.model_selection import learning_curve

# plot learning curve 

key_set = 'Al_tape_60nm_AuNPs'
train_sizes, train_scores, valid_scores = learning_curve(KNeighborsClassifier(), x_pca[key_set], y[key_set], train_sizes = np.linspace(0.1, 1.0, 5), scoring = 'accuracy', cv=int(K))
train_scores_mean = train_scores.mean(axis = 1)
valid_scores_mean = valid_scores.mean(axis = 1)
plt.figure(figsize = [15,10])
plt.plot(train_sizes, train_scores_mean, "o--", label = 'training scores')
plt.plot(train_sizes, valid_scores_mean, "o--", label = 'validation scores')
plt.legend()
plt.ylabel('accuracy')
plt.xlabel('training size')

# Decision boundary plots

Plot decision boundaries for PC1 and PC2 with various classifiers

In [ ]:
# create dict that contains only PC1 and PC2
x_pca_2 = {}

for key_set in x_pca.keys():
    x_pca_2[key_set] = x_pca[key_set][:,:2]

In [ ]:
no_keys = len(x_pca_2.keys())

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

# plot PC1 (x-axis) vs PC2 (y-axis)
plt.figure(figsize=[20,20])
for key_set, index_set in zip(x_pca_2.keys(), np.arange(1,no_keys+1)):
    
    x_axis = x_pca_2[key_set][:,0]
    y_axis = x_pca_2[key_set][:,1]
    colors=['red' if l==1 else 'blue' for l in y[key_set]]
    
#     # run classifier
#     classifier = LinearDiscriminantAnalysis().fit(x_pca_2[key_set], y[key_set])
    
#     disp = DecisionBoundaryDisplay.from_estimator(
#         classifier, x_pca_2[key_set], response_method="predict",
#         xlabel='PC1', ylabel='PC2',
#         alpha=0.5,
#     )
    
#     plt.title(key_set)
#     plt.xlim([np.min(x_axis),np.max(x_axis)])
#     plt.ylim([np.min(y_axis),np.max(y_axis)])
#     plt.figure(figsize=[8,8])
    
    
#     disp.ax_.scatter(x_axis, y_axis, color=colors)
    
    
    
    plt.subplot(3,3,index_set)
    plt.scatter(x_axis, y_axis, color=colors)
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.title(key_set)

In [ ]:
np.arange(0,7)

In [ ]:
np.arange(1,8)